In [ ]:
from utils.vector_db import vectorDataBase
import os
from langchain_google_genai import GoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.

# **LLM's**

In [10]:
llm = GoogleGenerativeAI(model="gemini-2.5-flash", verbose=True)

In [11]:
print(llm.invoke("Can you teach me bankai??"))

'Ah, a fellow Shinigami in training! You\'ve got your sights set on the ultimate release, the pinnacle of Zanpakuto power!\n\nFirst, let\'s get one thing straight: Bankai is a fictional concept from the anime and manga *Bleach*. While I can\'t *literally* teach you to manifest a spiritual weapon and achieve its final form, I can certainly explain how it\'s achieved within the lore of *Bleach*, and maybe we can find some real-world parallels in dedication and mastery!\n\n### How to Achieve Bankai (According to Bleach Lore)\n\nBankai (卍解, "Final Release" or "Full Release") is the second and final, much more powerful form of a Shinigami\'s Zanpakuto. It\'s incredibly difficult to achieve, and only a handful of Shinigami ever manage it.\n\nHere are the "steps" and requirements within the *Bleach* universe:\n\n1.  **Master Your Shikai:**\n    *   **Prerequisite:** You must first be able to fully understand and release your Zanpakuto\'s first stage, Shikai. This means knowing its name, its a

# **Vector Stores**

In [17]:
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001",
    task_type="retrieval_document"
)

db_creator = vectorDataBase()
pdf_store = db_creator.initalize_empty_db(
    emdeddingFunction=embeddings
)

Previous vector db is over written by this function
